In [1]:
import numpy as np 
import pandas as pd 
import re
import nltk
from matplotlib import pyplot as plt
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt 
from wordcloud import WordCloud, STOPWORDS 

In [ ]:

#https://www.kaggle.com/c/nlp-getting-started/overview

In [2]:
#Reading training and testing datasets 
ds_train=pd.read_csv("train.csv")
ds_test=pd.read_csv("test.csv")

In [3]:
#printing training dataset
ds_train.head()
print(len(ds_train))

7613


In [4]:
#Calculating NaN values.
train_nans= ds_train['keyword'].isnull().sum()
print(train_nans)
train_nans = ds_train['location'].isnull().sum()
print(train_nans)
test_nans= ds_test['keyword'].isnull().sum()
print(test_nans)
test_nans = ds_test['location'].isnull().sum()
print(test_nans)



61
2533
26
1105


In [5]:
# Replacing nans.
ds_train['keyword'].fillna('none', inplace=True)
ds_train['location'].fillna('Unknown', inplace=True)
ds_test['keyword'].fillna('none', inplace=True)
ds_test['location'].fillna('Unknown', inplace=True)

In [6]:
# Checking NaN values again.
ds_train.head()
train_nans= ds_train['keyword'].isnull().sum()
print(train_nans)
train_nans = ds_train['location'].isnull().sum()
print(train_nans)


0
0


In [7]:
print(ds_train.shape[0])

7613


In [8]:
# Creating Corpus after preprocessing the training data
corpus  = []
pstem = PorterStemmer()
for i in range(ds_train['text'].shape[0]):
    text = re.sub("[^a-zA-Z]", ' ', ds_train['text'][i])
    text = text.lower()
    text = text.split()
    text = [pstem.stem(word) for word in text if not word in set(stopwords.words('english'))]
    text = ' '.join(text)
    corpus.append(text)
    


In [9]:
#print((corpus))

In [10]:
corpus

['deed reason earthquak may allah forgiv us',
 'forest fire near la rong sask canada',
 'resid ask shelter place notifi offic evacu shelter place order expect',
 'peopl receiv wildfir evacu order california',
 'got sent photo rubi alaska smoke wildfir pour school',
 'rockyfir updat california hwi close direct due lake counti fire cafir wildfir',
 'flood disast heavi rain caus flash flood street manit colorado spring area',
 'top hill see fire wood',
 'emerg evacu happen build across street',
 'afraid tornado come area',
 'three peopl die heat wave far',
 'haha south tampa get flood hah wait second live south tampa gonna gonna fvck flood',
 'rain flood florida tampabay tampa day lost count',
 'flood bago myanmar arriv bago',
 'damag school bu multi car crash break',
 'man',
 'love fruit',
 'summer love',
 'car fast',
 'goooooooaaaaaal',
 'ridicul',
 'london cool',
 'love ski',
 'wonder day',
 'looooool',
 'way eat shit',
 'nyc last week',
 'love girlfriend',
 'cooool',
 'like pasta',
 '

In [11]:
#Create dictionary based on corpus 
uniqueWords = {}
for text in corpus:
    for word in text.split():
        
        if(word in uniqueWords.keys()):
            uniqueWords[word] = uniqueWords[word] + 1
        else:
            uniqueWords[word] = 1

In [12]:
print(uniqueWords)

{'deed': 2, 'reason': 31, 'earthquak': 53, 'may': 89, 'allah': 9, 'forgiv': 3, 'us': 133, 'forest': 67, 'fire': 363, 'near': 58, 'la': 28, 'rong': 1, 'sask': 1, 'canada': 13, 'resid': 9, 'ask': 28, 'shelter': 7, 'place': 35, 'notifi': 1, 'offic': 53, 'evacu': 131, 'order': 41, 'expect': 33, 'peopl': 201, 'receiv': 4, 'wildfir': 82, 'california': 121, 'got': 114, 'sent': 13, 'photo': 68, 'rubi': 2, 'alaska': 8, 'smoke': 54, 'pour': 4, 'school': 71, 'rockyfir': 4, 'updat': 57, 'hwi': 11, 'close': 40, 'direct': 18, 'due': 31, 'lake': 16, 'counti': 38, 'cafir': 2, 'flood': 169, 'disast': 162, 'heavi': 21, 'rain': 61, 'caus': 106, 'flash': 21, 'street': 32, 'manit': 1, 'colorado': 16, 'spring': 18, 'area': 59, 'top': 58, 'hill': 12, 'see': 128, 'wood': 6, 'emerg': 164, 'happen': 56, 'build': 153, 'across': 20, 'afraid': 5, 'tornado': 42, 'come': 125, 'three': 31, 'die': 82, 'heat': 49, 'wave': 76, 'far': 28, 'haha': 20, 'south': 32, 'tampa': 4, 'get': 311, 'hah': 3, 'wait': 39, 'second': 36

In [13]:
print(len(uniqueWords))

18889


In [14]:
#Converting dictionary to dataFrame
uniqueWords = pd.DataFrame.from_dict(uniqueWords,orient='index',columns=['WordFrequency'])
uniqueWords.sort_values(by=['WordFrequency'], inplace=True, ascending=False)
#print(uniqueWords)
print("Number of records in Unique Words Data frame are ")
print((len(uniqueWords)))
uniqueWords.head(10)

Number of records in Unique Words Data frame are 
18889


,WordFrequency
co,4746
http,4721
like,411
fire,363
amp,344
get,311
bomb,239
new,228
via,220
u,216


In [17]:
# Creating the Bag of Words model by vectorizing the input data
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = len(uniqueWords))
X = cv.fit_transform(corpus).todense()
y = ds_train['target'].values

In [18]:
print(X.shape[0])

7613


In [19]:
#Split the training data set to train and test data
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.2)
print('Train Data splitted successfully')
print(X_test)

Train Data splitted successfully
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [20]:
from sklearn.neighbors import KNeighborsClassifier

In [22]:
# Applying Model KNN
classifier_knn = KNeighborsClassifier(n_neighbors = 6,weights = 'distance',algorithm = 'brute')
classifier_knn.fit(X_train, y_train)
y_pred_knn = classifier_knn.predict(X_test)


In [30]:
#Calculating Evaluation Measures
print("K-Nearest Neighbour Model Accuracy Score for Train Data set is")
print((classifier_knn.score(X_train, y_train)))
print("K-Nearest Neighbour Model Accuracy Score for Test Data set is ")
print(classifier_knn.score(X_test, y_test) )   
print("K-Nearest Neighbour Model F1 Score is ")
print((f1_score(y_test, y_pred_knn)))


K-Nearest Neighbour Model Accuracy Score for Train Data set is
0.9963875205254515
K-Nearest Neighbour Model Accuracy Score for Test Data set is 
0.6900853578463558
K-Nearest Neighbour Model F1 Score is 
0.48131868131868133
